In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split

insurance = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_fraud.csv', on_bad_lines='skip')

# Ordinal feature encoding
# https://www.kaggle.com/pratik1120/penguin-dataset-eda-classification-and-clustering
df = insurance.copy()


# Separating X and y
X = df.drop('fraud_reported', axis=1)
Y = df['fraud_reported']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
        
        
# Build random forest model
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)


# Saving the model
"""
import pickle
pickle.dump(knn, open('insurance.pkl', 'wb'))
"""

"\nimport pickle\npickle.dump(knn, open('insurance.pkl', 'wb'))\n"

In [54]:
#ça devrait donner 1, fraud
a = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/clean_data_example.csv', on_bad_lines='skip')
a

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1406.91,0,53300,0,1,5,1,1,13020


In [56]:
knn.predict(a)

array([1], dtype=int64)

In [57]:
#ça devrait donner 0, pas de fraud
b = pd.read_csv('https://raw.githubusercontent.com/oualid-ben/data/main/example_knn.csv', on_bad_lines='skip')
b

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1442.99,0,0,0,2,21,1,1,2770


In [58]:
knn.predict(b)

array([0], dtype=int64)

In [59]:
import pickle
import numpy as np

In [124]:
load_clf = pickle.load(open('insurance.pkl', 'rb'))

In [125]:
prediction = load_clf.predict(a)
prediction_proba = load_clf.predict_proba(a)

In [126]:
prediction.item(0)

1

In [127]:
prediction_proba

array([[0.33333333, 0.66666667]])

In [128]:
insurance_species = np.array(['No fraud','Yes, fraud'])
insurance_species[prediction.item(0)]

'Yes, fraud'

In [85]:
from sklearn import svm

In [86]:
# Build SVN model
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [87]:
a

,policy_annual_premium,umbrella_limit,capital_gains,capital_loss,incident_severity,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,property_claim
0,1406.91,0,53300,0,1,5,1,1,13020


In [88]:
clf.predict(a)

array([0], dtype=int64)

In [89]:
clf.predict(b)

array([0], dtype=int64)

In [130]:
sm = pickle.load(open('insurance_svm.pkl', 'rb'))
pred = sm.predict(a)
proba = sm.predict_proba(a)

In [131]:
pred

array([0], dtype=int64)

In [132]:
proba

array([[0.74931201, 0.25068799]])

In [133]:
insurance_species[pred.item(0)]

'No fraud'